In [1]:
begin 
    using CSV 
    using DataFrames 
    using DataFrameMacros
    using GLMakie; GLMakie.activate!(ssao=true)
    # using Plots 
    # using CairoMakie
end

In [6]:
pnames = ["ka1","kb1","kcat1"]

# load CSV into DataFrame 
df = CSV.read(pwd()*"/3FixedResultsCSVs/fixed_triplet_results-$(pnames[1])$(pnames[2])$(pnames[3]).csv", DataFrame)
excessL_df = CSV.read(pwd()*"/3FixedResultsCSVsExcessL/fixed_triplet_results-$(pnames[1])$(pnames[2])$(pnames[3]).csv", DataFrame)

Row,ka1,kb1,kcat1,num_oscillatory_points,average_period,maximum_period,minimum_period,average_amplitude,maximum_amplitude,minimum_amplitude
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.001,0.01,0.01,0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.001,0.01,0.464159,3173,340.14,1005.4,0.0,-0.0196956,0.208948,-0.165905
3,0.001,0.01,21.5443,5778,93.7073,640.1,0.0,-0.168171,0.311765,-0.431517
4,0.001,0.01,1000.0,10228,118.605,830.8,0.0,-0.203123,0.465232,-0.557061
5,0.001,0.464159,0.01,0,NaN,NaN,NaN,NaN,NaN,NaN
6,0.001,0.464159,0.464159,3001,436.495,1055.8,0.0,0.00221336,0.556656,-0.40818
7,0.001,0.464159,21.5443,8009,68.8508,737.6,0.0,-0.115762,0.679197,-0.525773
8,0.001,0.464159,1000.0,13699,101.53,924.4,0.0,-0.233293,0.467003,-0.686618
9,0.001,21.5443,0.01,5013,471.073,610.5,0.0,-0.471052,0.0,-0.870057


In [8]:
#< GLMakie plots

function scatters_in_3D(df; fig = Figure(resolution=(1600, 1000)),  angle=30)
    pnames = names(df)[1:3]
    xlog = log10.(df[:, pnames[1]])
    ylog = log10.(df[:, pnames[2]])
    zlog = log10.(df[:, pnames[3]])

    x = df[:, pnames[1]]
    y = df[:, pnames[2]]
    z = df[:, pnames[3]]

    # Identify non-NaN indices and values
    nonan_indices = findall(!isnan, df[:, :average_period])
    # nonan_amplitudes = df[:, :average_amplitude][nonan_indices]
    nonan_numpoints = df[:, :num_oscillatory_points][nonan_indices]

    nonan_periods = df[:, :average_period][nonan_indices]

    # Normalize sizes for non-NaN values
    sizes = fill(0.1, size(df, 1))
    # sizes[nonan_indices] = ((nonan_amplitudes .- minimum(nonan_amplitudes)) ./ (maximum(nonan_amplitudes) - minimum(nonan_amplitudes))) ./ 2 
    sizes[nonan_indices] = ((nonan_numpoints .- minimum(nonan_numpoints)) ./ (maximum(nonan_numpoints) - minimum(nonan_numpoints))) ./ 2 


    # Normalize periods for non-NaN values
    norm_periods = fill(NaN, size(df, 1))

    norm_periods[nonan_indices] = (nonan_periods .- minimum(nonan_periods)) ./ (maximum(nonan_periods) - minimum(nonan_periods)) 

    # Create the figure and axis
    
    ax = Axis3(fig[1:2,1]; aspect=:data, perspectiveness=0.5, title="3 Fixed Parameter Oscillatory Regions", xlabel = pnames[1], ylabel = pnames[2], zlabel = pnames[3])

    # Scatter plot for non-NaN values
    hm = meshscatter!(ax, xlog, ylog, zlog; markersize=sizes, ssao=true, color=df.average_period, colormap=:thermal, transparency=false, nan_color=:gray,
                        diffuse = Vec3f(0.5, 0.5, 0.5), specular = Vec3f(0.3, 0.3, 0.3), shininess = 100f0, ambient = Vec3f(0.1), shading=true)

    # meshscatter!(ax, xlog, ylog; markersize=sizes, marker= Rect3f(Vec3f(0.,0.,0.1)), ssao=true, color=df.average_period, colormap=:thermal, transparency=false, nan_color=:gray,
    #                     diffuse = Vec3f(0.0), specular = Vec3f(0.0), shininess = 0, ambient = Vec3f(0.0))

    # Scatter plot for NaN values in gray
    # nan_indices = findall(isnan, df[:, :average_period])
    # meshscatter!(ax3, x[nan_indices], y[nan_indices], z[nan_indices]; markersize=sizes[nan_indices], color=:gray)

    # Colorbar and labels
    Colorbar(fig[2, 4], hm, label="Period (s)", height=Relative(2.0))
    colgap!(fig.layout, 5)
    # xlabel!(ax3, "log10(kb3)")
    # ylabel!(ax3, "log10(kb4)")
    # zlabel!(ax3, "log10(DF)")

    # Display plot
    fig
end

scatters_in_3D (generic function with 1 method)

In [9]:
fig = scatters_in_3D(df)

# fig2 = scatters_in_3D(excessL_df; fig=fig)